In [1]:
import pandas as pd
import numpy as np
import sklearn
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
import pandas.core.algorithms as algos
from pandas import Series
import scipy.stats.stats as stats
import re
import traceback
import string
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor

# General Information about Data Sets

In [2]:
train2= pd.read_csv('cleaned_train.csv')
weather= pd.read_csv('cleaned_weather.csv')
spray2= pd.read_csv('cleaned_spray.csv')
train= train2.drop(['Year','Month','Day'], axis=1)
spray= spray2.drop(['Year','YearMonth','Day','YearWeek','Month'], axis=1)
train['Date']=pd.to_datetime(train['Date'])
spray['Date']=pd.to_datetime(spray['Date'])
weather['Date']=pd.to_datetime(weather['Date'])
weather['YearWeek']= (weather['Year'].astype(str)+weather['Week'].astype(str)).astype('int64')
weather['YearMonth']=(weather['Year'].astype(str)+weather['Month'].astype(str)).astype('int64')
weather.drop(['Week','Day'], axis=1, inplace=True)

In [3]:
print(train.info(),weather.info(),spray.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9693 entries, 0 to 9692
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   Date          9693 non-null   datetime64[ns]
 1   Species       9693 non-null   object        
 2   Latitude      9693 non-null   float64       
 3   Longitude     9693 non-null   float64       
 4   NumMosquitos  9693 non-null   int64         
 5   WnvPresent    9693 non-null   int64         
dtypes: datetime64[ns](1), float64(2), int64(2), object(1)
memory usage: 454.5+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1472 entries, 0 to 1471
Data columns (total 19 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   Date         1472 non-null   datetime64[ns]
 1   Tmax         1472 non-null   int64         
 2   Tmin         1472 non-null   int64         
 3   Tavg         1472 non-null   int64         
 4   Depar

In [4]:
train.head()

,Date,Species,Latitude,Longitude,NumMosquitos,WnvPresent
0,2007-05-29,CULEX PIPIENS/RESTUANS,41.954690,-87.800991,1,0
1,2007-05-29,CULEX RESTUANS,41.954690,-87.800991,1,0
2,2007-05-29,CULEX RESTUANS,41.994991,-87.769279,1,0
3,2007-05-29,CULEX PIPIENS/RESTUANS,41.974089,-87.824812,1,0
4,2007-05-29,CULEX RESTUANS,41.974089,-87.824812,4,0


In [5]:
spray.head()

,Date,Latitude,Longitude
0,2011-08-29,42.391623,-88.089163
1,2011-08-29,42.391348,-88.089163
2,2011-08-29,42.391022,-88.089157
3,2011-08-29,42.390637,-88.089158
4,2011-08-29,42.390410,-88.088858


In [6]:
weather.head()

,Date,Tmax,Tmin,Tavg,Depart,DewPoint,WetBulb,Heat,Cool,PrecipTotal,StnPressure,SeaLevel,ResultSpeed,ResultDir,AvgSpeed,Year,Month,YearWeek,YearMonth
0,2007-05-01,83,51,67,14.0,51.0,56.5,0.0,2.5,0.0,29.140,29.820,2.20,26.0,9.40,2007,5,200718,20075
1,2007-05-02,59,42,51,-3.0,42.0,47.0,13.5,0.0,0.0,29.410,30.085,13.15,3.0,13.40,2007,5,200718,20075
2,2007-05-03,66,47,57,2.0,40.0,49.0,8.0,0.0,0.0,29.425,30.120,12.30,6.5,12.55,2007,5,200718,20075
3,2007-05-04,72,50,61,4.0,41.5,50.0,4.0,0.0,0.0,29.335,30.045,10.25,7.5,10.60,2007,5,200718,20075
4,2007-05-05,66,53,60,5.0,38.5,49.5,5.0,0.0,0.0,29.430,30.095,11.45,7.0,11.75,2007,5,200718,20075


# Weather & Train Data Sets - EDA and Feature Engineering

According to the researches, if the weather gets too hot and too dry, mosquitoes will not be as active and feeding as they usually are. But once the humidity increases they’re more hungry and biting more.Therefore humidity plays a key role in WVN Presence. We will add the relative humidity as a feature.

In [7]:
# calculation of RELATIVE HUMIDITY
def rel_humidity(df,T, Td,Tw):
    
# Convert the air temperature and dew-point temperature to Celsius.(C=5*(F-32)/9)
    Tc= (5.0*(df[T]-32))/9.0
    Tdc= (5.0*(df[Td]-32))/9.0

#Calculate the saturated vapor pressure with a formula.  
    es=6.11*10.0**(7.5*Tc/(237.7+Tc))

#Find the actual vapor pressure with the same formula.
    e=6.11*10.0**(7.5*Tdc/(237.7+Tdc))
    
#Calculate the relative humidity.    
    df['RelHumidity']= round((e/es)*100)
    return df

In [8]:
rel_humidity(weather,'Tavg','DewPoint', 'WetBulb')

,Date,Tmax,Tmin,Tavg,Depart,DewPoint,WetBulb,Heat,Cool,PrecipTotal,StnPressure,SeaLevel,ResultSpeed,ResultDir,AvgSpeed,Year,Month,YearWeek,YearMonth,RelHumidity
0,2007-05-01,83,51,67,14.0,51.0,56.5,0.0,2.5,0.000,29.140,29.820,2.20,26.0,9.40,2007,5,200718,20075,56.0
1,2007-05-02,59,42,51,-3.0,42.0,47.0,13.5,0.0,0.000,29.410,30.085,13.15,3.0,13.40,2007,5,200718,20075,71.0
2,2007-05-03,66,47,57,2.0,40.0,49.0,8.0,0.0,0.000,29.425,30.120,12.30,6.5,12.55,2007,5,200718,20075,53.0
3,2007-05-04,72,50,61,4.0,41.5,50.0,4.0,0.0,0.000,29.335,30.045,10.25,7.5,10.60,2007,5,200718,20075,49.0
4,2007-05-05,66,53,60,5.0,38.5,49.5,5.0,0.0,0.000,29.430,30.095,11.45,7.0,11.75,2007,5,200718,20075,45.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1467,2014-10-27,78,52,65,16.0,51.5,58.5,0.5,1.0,0.010,28.960,29.665,12.35,19.0,13.25,2014,10,201444,201410,62.0
1468,2014-10-28,67,46,57,10.0,39.0,47.5,8.0,0.0,0.025,29.190,29.850,14.40,26.0,15.10,2014,10,201444,201410,51.0
1469,2014-10-29,49,38,44,-4.0,33.0,41.0,21.0,0.0,0.000,29.390,30.065,9.00,29.0,9.45,2014,10,201444,201410,65.0
1470,2014-10-30,52,34,43,-4.0,34.5,41.0,21.5,0.0,0.000,29.375,30.095,5.50,23.5,6.00,2014,10,201444,201410,72.0


I will calculate time lags by giving higher weights to the most recent observed values.

def ema(df,col, span):
    df[f'{col}_{span}']= round(df[col].ewm(span=span, adjust=True).mean(),2)
    return df
ema_list= ['Tmax', 'DewPoint','WetBulb','PrecipTotal', 'StnPressure','SeaLevel', 'ResultSpeed', 'ResultDir', 'AvgSpeed','RelHumidity']
span= [7,14]
for i in ema_list:
    ema(weather,i, 7)
    ema(weather,i, 14)

In [9]:
def ema(df,col, span):
    df[f'{col}_{span}']= round(df[col].ewm(span=span, adjust=True).mean(),2)
    return df
ema_list= ['Tmax', 'DewPoint','WetBulb','PrecipTotal', 'StnPressure','SeaLevel', 'ResultSpeed', 'ResultDir', 'AvgSpeed','RelHumidity']
span= list(range(1,15))
for i in ema_list:
    for s in span:
        ema(weather,i,s)

In [10]:
weather.head()

,Date,Tmax,Tmin,Tavg,Depart,DewPoint,WetBulb,Heat,Cool,PrecipTotal,...,RelHumidity_5,RelHumidity_6,RelHumidity_7,RelHumidity_8,RelHumidity_9,RelHumidity_10,RelHumidity_11,RelHumidity_12,RelHumidity_13,RelHumidity_14
0,2007-05-01,83,51,67,14.0,51.0,56.5,0.0,2.5,0.0,...,56.00,56.00,56.00,56.00,56.00,56.00,56.00,56.00,56.00,56.00
1,2007-05-02,59,42,51,-3.0,42.0,47.0,13.5,0.0,0.0,...,65.00,64.75,64.57,64.44,64.33,64.25,64.18,64.12,64.08,64.04
2,2007-05-03,66,47,57,2.0,40.0,49.0,8.0,0.0,0.0,...,59.32,59.47,59.57,59.64,59.69,59.73,59.76,59.78,59.80,59.82
3,2007-05-04,72,50,61,4.0,41.5,50.0,4.0,0.0,0.0,...,55.03,55.42,55.70,55.91,56.07,56.19,56.30,56.38,56.45,56.51
4,2007-05-05,66,53,60,5.0,38.5,49.5,5.0,0.0,0.0,...,51.18,51.77,52.19,52.52,52.78,52.98,53.15,53.29,53.41,53.51


In [11]:
weather.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1472 entries, 0 to 1471
Columns: 160 entries, Date to RelHumidity_14
dtypes: datetime64[ns](1), float64(152), int64(7)
memory usage: 1.8 MB


In [12]:
dur dur    [sx;slcdcd;]

SyntaxError: invalid syntax (<ipython-input-12-b5ea9fe6ed96>, line 1)

In [13]:
w_train= train.merge(weather, on= ['Date'])
w_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9693 entries, 0 to 9692
Columns: 165 entries, Date to RelHumidity_14
dtypes: datetime64[ns](1), float64(154), int64(9), object(1)
memory usage: 12.3+ MB


**The below chart indicates CULEX RESTUANS species were most affected by temperature increases.**

In [ ]:
Virus_Species_Month=w_train[w_train['WnvPresent']==1].groupby(['Tmax'])['Species'].value_counts().unstack().fillna(0)
Virus_Species_Month.plot.bar(figsize=(14,6))
plt.grid(False)
plt.legend(bbox_to_anchor=(1, 1), fontsize='small')
plt.xlabel('Max Temperature', fontsize=13)
plt.ylabel('Number of WN Virus', fontsize=13)
plt.title('WN Virus Distribution per Species and Maximum Temperature', fontsize=18, color='red')
plt.show() 

In [14]:
w_train=pd.get_dummies(w_train,drop_first=True)
w_train.head()

,Date,Latitude,Longitude,NumMosquitos,WnvPresent,Tmax,Tmin,Tavg,Depart,DewPoint,...,RelHumidity_11,RelHumidity_12,RelHumidity_13,RelHumidity_14,Species_CULEX PIPIENS,Species_CULEX PIPIENS/RESTUANS,Species_CULEX RESTUANS,Species_CULEX SALINARIUS,Species_CULEX TARSALIS,Species_CULEX TERRITANS
0,2007-05-29,41.954690,-87.800991,1,0,88,62,75,10.0,58.5,...,54.88,54.61,54.37,54.16,0,1,0,0,0,0
1,2007-05-29,41.954690,-87.800991,1,0,88,62,75,10.0,58.5,...,54.88,54.61,54.37,54.16,0,0,1,0,0,0
2,2007-05-29,41.994991,-87.769279,1,0,88,62,75,10.0,58.5,...,54.88,54.61,54.37,54.16,0,0,1,0,0,0
3,2007-05-29,41.974089,-87.824812,1,0,88,62,75,10.0,58.5,...,54.88,54.61,54.37,54.16,0,1,0,0,0,0
4,2007-05-29,41.974089,-87.824812,4,0,88,62,75,10.0,58.5,...,54.88,54.61,54.37,54.16,0,0,1,0,0,0


In [15]:
w_train[w_train.duplicated()]

,Date,Latitude,Longitude,NumMosquitos,WnvPresent,Tmax,Tmin,Tavg,Depart,DewPoint,...,RelHumidity_11,RelHumidity_12,RelHumidity_13,RelHumidity_14,Species_CULEX PIPIENS,Species_CULEX PIPIENS/RESTUANS,Species_CULEX RESTUANS,Species_CULEX SALINARIUS,Species_CULEX TARSALIS,Species_CULEX TERRITANS


In [ ]:
print('\n','The Percantage of Virus per Each Species  ','\n')
print('CULEX PIPIENS: ', round(w_train[w_train.WnvPresent==1]['Species_CULEX PIPIENS'].sum()/len(w_train[w_train.WnvPresent==1]['Species_CULEX PIPIENS'])*100,2))
print('CULEX RESTUANS: ', round(w_train[w_train.WnvPresent==1]['Species_CULEX RESTUANS'].sum()/len(w_train[w_train.WnvPresent==1]['Species_CULEX RESTUANS'])*100,2))
print('CULEX PIPIENS/RESTUANS: ', round(w_train[w_train.WnvPresent==1]['Species_CULEX PIPIENS/RESTUANS'].sum()/len(w_train[w_train.WnvPresent==1]['Species_CULEX PIPIENS/RESTUANS'])*100),2)

**The below Correlation report indicates Relative Humidity is highly correlated with WN Virus, consequently Dew Temperature and Wet Bulb. Also their 7 and 14 days Exponential Mean Averages have higher correlation than the actual observation values. This must be because of the incubation period.**

In [ ]:
figure = plt.figure(figsize=(10,40))
sns.heatmap(w_train.corr()[['WnvPresent']].sort_values('WnvPresent',ascending=False),annot=True, cmap='YlGnBu')

**We will check if there is multicollinearity among the features and fix them.**

**WHY SO HIGH ???**

In [16]:
pd.options.mode.use_inf_as_na = True

max_bin = 20
force_bin = 3

# define a binning function
def mono_bin(Y, X, n = max_bin):
    
    df1 = pd.DataFrame({"X": X, "Y": Y})
    justmiss = df1[['X','Y']][df1.X.isnull()]
    notmiss = df1[['X','Y']][df1.X.notnull()]
    r = 0
    while np.abs(r) < 1:
        try:
            d1 = pd.DataFrame({"X": notmiss.X, "Y": notmiss.Y, "Bucket": pd.qcut(notmiss.X, n)})
            d2 = d1.groupby('Bucket', as_index=True)
            r, p = stats.spearmanr(d2.mean().X, d2.mean().Y)
            n = n - 1 
        except Exception as e:
            n = n - 1

    if len(d2) == 1:
        n = force_bin         
        bins = algos.quantile(notmiss.X, np.linspace(0, 1, n))
        if len(np.unique(bins)) == 2:
            bins = np.insert(bins, 0, 1)
            bins[1] = bins[1]-(bins[1]/2)
        d1 = pd.DataFrame({"X": notmiss.X, "Y": notmiss.Y, "Bucket": pd.cut(notmiss.X, np.unique(bins),include_lowest=True)}) 
        d2 = d1.groupby('Bucket', as_index=True)
    
    d3 = pd.DataFrame({},index=[])
    d3["MIN_VALUE"] = d2.min().X
    d3["MAX_VALUE"] = d2.max().X
    d3["COUNT"] = d2.count().Y
    d3["EVENT"] = d2.sum().Y
    d3["NONEVENT"] = d2.count().Y - d2.sum().Y
    d3=d3.reset_index(drop=True)
    
    if len(justmiss.index) > 0:
        d4 = pd.DataFrame({'MIN_VALUE':np.nan},index=[0])
        d4["MAX_VALUE"] = np.nan
        d4["COUNT"] = justmiss.count().Y
        d4["EVENT"] = justmiss.sum().Y
        d4["NONEVENT"] = justmiss.count().Y - justmiss.sum().Y
        d3 = d3.append(d4,ignore_index=True)
    
    d3["EVENT_RATE"] = d3.EVENT/d3.COUNT
    d3["NON_EVENT_RATE"] = d3.NONEVENT/d3.COUNT
    d3["DIST_EVENT"] = d3.EVENT/d3.sum().EVENT
    d3["DIST_NON_EVENT"] = d3.NONEVENT/d3.sum().NONEVENT
    d3["WOE"] = np.log(d3.DIST_EVENT/d3.DIST_NON_EVENT)
    d3["IV"] = (d3.DIST_EVENT-d3.DIST_NON_EVENT)*np.log(d3.DIST_EVENT/d3.DIST_NON_EVENT)
    d3["VAR_NAME"] = "VAR"
    d3 = d3[['VAR_NAME','MIN_VALUE', 'MAX_VALUE', 'COUNT', 'EVENT', 'EVENT_RATE', 'NONEVENT', 'NON_EVENT_RATE', 'DIST_EVENT','DIST_NON_EVENT','WOE', 'IV']]       
    d3 = d3.replace([np.inf, -np.inf], 0)
    d3.IV = d3.IV.sum()
    
    return(d3)

def char_bin(Y, X):
        
    df1 = pd.DataFrame({"X": X, "Y": Y})
    justmiss = df1[['X','Y']][df1.X.isnull()]
    notmiss = df1[['X','Y']][df1.X.notnull()]    
    df2 = notmiss.groupby('X',as_index=True)
    
    d3 = pd.DataFrame({},index=[])
    d3["COUNT"] = df2.count().Y
    d3["MIN_VALUE"] = df2.sum().Y.index
    d3["MAX_VALUE"] = d3["MIN_VALUE"]
    d3["EVENT"] = df2.sum().Y
    d3["NONEVENT"] = df2.count().Y - df2.sum().Y
    
    if len(justmiss.index) > 0:
        d4 = pd.DataFrame({'MIN_VALUE':np.nan},index=[0])
        d4["MAX_VALUE"] = np.nan
        d4["COUNT"] = justmiss.count().Y
        d4["EVENT"] = justmiss.sum().Y
        d4["NONEVENT"] = justmiss.count().Y - justmiss.sum().Y
        d3 = d3.append(d4,ignore_index=True)
    
    d3["EVENT_RATE"] = d3.EVENT/d3.COUNT
    d3["NON_EVENT_RATE"] = d3.NONEVENT/d3.COUNT
    d3["DIST_EVENT"] = d3.EVENT/d3.sum().EVENT
    d3["DIST_NON_EVENT"] = d3.NONEVENT/d3.sum().NONEVENT
    d3["WOE"] = np.log(d3.DIST_EVENT/d3.DIST_NON_EVENT)
    d3["IV"] = (d3.DIST_EVENT-d3.DIST_NON_EVENT)*np.log(d3.DIST_EVENT/d3.DIST_NON_EVENT)
    d3["VAR_NAME"] = "VAR"
    d3 = d3[['VAR_NAME','MIN_VALUE', 'MAX_VALUE', 'COUNT', 'EVENT', 'EVENT_RATE', 'NONEVENT', 'NON_EVENT_RATE', 'DIST_EVENT','DIST_NON_EVENT','WOE', 'IV']]      
    d3 = d3.replace([np.inf, -np.inf], 0)
    d3.IV = d3.IV.sum()
    d3 = d3.reset_index(drop=True)
    
    return(d3)

def data_vars(df1, target):
    
    stack = traceback.extract_stack()
    filename, lineno, function_name, code = stack[-2]
    vars_name = re.compile(r'\((.*?)\).*$').search(code).groups()[0]
    final = (re.findall(r"[\w']+", vars_name))[-1]
    
    x = df1.dtypes.index
    count = -1
    
    for i in x:
        #print(i)
        if i.upper() not in (final.upper()):
            if np.issubdtype(df1[i], np.number) and len(pd.Series.unique(df1[i])) > 2:
                conv = mono_bin(target, df1[i])
                conv["VAR_NAME"] = i
                count = count + 1
            else:
                conv = char_bin(target, df1[i])
                conv["VAR_NAME"] = i            
                count = count + 1
                
            if count == 0:
                iv_df = conv
            else:
                iv_df = iv_df.append(conv,ignore_index=True)
    
    iv = pd.DataFrame({'IV':iv_df.groupby('VAR_NAME').IV.max()})
    iv = iv.reset_index()
    return(iv_df,iv)

In [17]:
X = w_train.drop(['WnvPresent'], axis=1)
y = w_train[['WnvPresent']]
X_train,X_test, y_train,y_test = train_test_split(X, y,test_size=0.3)
X_train['Date'].value_counts()
X_train[X_train.duplicated()]
X.drop_duplicates(inplace=True)
y_train[y_train.duplicated()]
y.drop_duplicates(inplace=True)
print(X_train.shape, y_train.shape)
#final_iv, IV = data_vars(X_train, y_train)

(6785, 169) (6785, 1)


C:\Users\laptop\anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [19]:
df_ivs, IV = data_vars(X_train,y_train)

ValueError: Shape of passed values is (1, 2), indices imply (6785, 2)

In [ ]:
# w_train.drop(['Date'], axis=1, inplace=True)
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor

vif = pd.DataFrame()
vif["VIF Factor"] = [int(variance_inflation_factor(w_train.values, i)) for i in range(len(w_train.columns))]
vif["features"] = w_train.columns
vif

# Spray & Train Data

**Virus have been found in most of the trap locations. However spraying doesn't cover the entre trap and virus locations.**

In [ ]:
from sklearn.neighbors import KernelDensity
mapdata = np.loadtxt("mapdata_copyright_openstreetmap_contributors.txt")
traps = train[['Date','Longitude', 'Latitude', 'WnvPresent','NumMosquitos']]

alpha_cm = plt.cm.Reds
alpha_cm._init()
alpha_cm._lut[:-3,-1] = abs(np.logspace(0, 1, alpha_cm.N) / 10 - 1)[::-1]
aspect = mapdata.shape[0] * 1.0 / mapdata.shape[1]
lon_lat_box = (-88, -87.5, 41.6, 42.1)

spray_loc= spray[['Longitude', 'Latitude']].values
kd = KernelDensity(bandwidth=0.010)
kd.fit(spray_loc)

xv,yv = np.meshgrid(np.linspace(-88, -87.5, 100), np.linspace(41.6, 42.1, 100))
gridpoints = np.array([xv.ravel(),yv.ravel()]).T
zv = np.exp(kd.score_samples(gridpoints).reshape(100,100))

plt.figure(figsize=(18,14))
plt.imshow(mapdata, cmap=plt.get_cmap('gray'), extent=lon_lat_box, aspect=aspect)
plt.imshow(zv,origin='lower',cmap=alpha_cm, extent=lon_lat_box, aspect=aspect)

virus_loc = train[train['WnvPresent']==1][['Longitude', 'Latitude']].drop_duplicates().values
plt.scatter(virus_loc[:,0], virus_loc[:,1], marker='o', color='yellow', label='Virus Locations')
trap_loc = train[['Longitude', 'Latitude']].drop_duplicates().values
plt.scatter(trap_loc[:,0], trap_loc[:,1], marker='.', color='green', label='Trap Locations')
plt.legend(fontsize=15)
plt.title('Locations of Sprayed Traps and Virus', fontsize=22, color='red')
plt.xlabel('Longitude', fontsize=15)
plt.ylabel('Latitude', fontsize=15)
plt.show()